# Linearized Dynamics Model with a Kalman Filter

This notebook simulates a differential drive (tank drive) robot with noisy measurements and state estimation using a Kalman filter. The robot has two independently controlled wheels, allowing it to drive forward/backward and turn by varying the speeds of the left and right wheels.

## Overview

The simulation models:
- **Robot dynamics**: A linearized kinematic model with first-order wheel speed dynamics
- **Process noise**: Uncertainty in the true system evolution
- **Measurement noise**: Sensor uncertainty
- **State estimation**: A discrete-time Kalman filter to estimate the robot's state from noisy measurements

## Imports

In [ ]:
import sys
sys.path.insert(0, "..")  # Add parent directory for module imports

import numpy as np
import matplotlib.pyplot as plt
import control as ctl
from kalman import KalmanFilter
from lqr_controller import SwitchingLQRController
from simulation import TankDriveSimulation, discretize_system

## Parameters

### Physical Parameters
- **$L$**: Wheelbase (distance between left and right wheels) = 0.5 m
- **$\tau$**: Time constant for wheel speed dynamics = 0.2 s
  - Models the lag between commanded and actual wheel speeds (motor + drivetrain dynamics)
- **$v_0$**: Linearization point for forward speed = 0.5 m/s
  - Used to linearize the nonlinear kinematics around a nominal forward velocity

### State Vector
The robot state is: $\mathbf{x} = [x, y, \theta, v_l, v_r]^T$ where:
- $x, y$: Position in the global frame (m)
- $\theta$: Heading angle (rad)
- $v_l, v_r$: Left and right wheel speeds (m/s)

### Input Vector
The control inputs are: $\mathbf{u} = [u_l, u_r]^T$ (commanded wheel speeds)

### Noise Covariances
- **$Q$**: Process noise covariance (models uncertainty in state evolution)
- **$R$**: Measurement noise covariance (models sensor uncertainty)

In [ ]:
# Physical parameters
L = 0.5       # wheelbase [m]
tau = 0.2     # wheel speed time constant [s]
v0 = 0.5      # linearization forward speed [m/s]

# State: [x, y, theta, v_l, v_r]
n_states = 5
n_inputs = 2
n_outputs = 4   # realistic sensor measurements: [theta, theta_dot, v_l, v_r]

# Simulation parameters
T_final = 15.0
dt = 0.02   # 50 Hz update
N = int(T_final / dt)
t = np.arange(N) * dt
random_seed = 42

# Noise covariances
Q = np.diag([1e-5, 1e-5, 1e-6, 1e-4, 1e-4])  # process noise
# Measurement noise: [theta (mag), theta_dot (gyro), v_l (encoder), v_r (encoder)]
R = np.diag([1e-3, 1e-2, 1e-3, 1e-3])  # measurement noise

# Initial state covariance
P0 = 0.1  # initial uncertainty (scaled identity matrix)

# LQR weight matrices
Q_lqr = np.diag([10.0, 10.0, 5.0, 0.1, 0.1])  # state weights: [x, y, theta, v_l, v_r]
R_lqr = np.diag([1.0, 1.0])  # control weights: [u_l, u_r]

# Control parameters
u_max = 2.0  # control saturation limit [m/s]

# Target parameters
x_target = 5.0  # target x position [m]
y_target = 3.0  # target y position [m]
target_threshold = 0.5  # distance threshold for switching to stop mode [m]

# Reference states
x_ref_pos = np.array([x_target, y_target, 0.0, v0, v0])
x_ref_stop = np.array([x_target, y_target, 0.0, 0.0, 0.0])

## System Model and Equations of Motion

### Continuous-Time State-Space Model

The robot dynamics are represented in state-space form:

$$\dot{\mathbf{x}} = A\mathbf{x} + B\mathbf{u}$$
$$\mathbf{y} = C\mathbf{x} + D\mathbf{u}$$

### State Matrix $A$

The linearized dynamics capture:
1. **Position rate** ($\dot{x}$): Average of left and right wheel speeds
   - $\dot{x} = \frac{v_l + v_r}{2} \cos(\theta) \approx \frac{v_l + v_r}{2}$ (linearized at $\theta = 0$)

2. **Lateral motion** ($\dot{y}$): Coupled to heading and forward velocity
   - $\dot{y} = v_0 \theta$ (linearized around small angles)

3. **Angular rate** ($\dot{\theta}$): Difference in wheel speeds divided by wheelbase
   - $\dot{\theta} = \frac{v_r - v_l}{L}$

4. **Wheel dynamics**: First-order lag with time constant $\tau$
   - $\dot{v}_l = -\frac{1}{\tau}v_l + \frac{1}{\tau}u_l$
   - $\dot{v}_r = -\frac{1}{\tau}v_r + \frac{1}{\tau}u_r$

### Input Matrix $B$

Maps commanded wheel speeds to state derivatives. Only the wheel speed states are directly affected by inputs.

### Output Matrices $C$ and $D$

**Realistic sensor model** based on available hardware:
- **Magnetometer**: Measures heading $\theta$
- **Gyroscope**: Measures angular rate $\dot{\theta} = \frac{v_r - v_l}{L}$
- **Wheel encoders**: Measure left and right wheel speeds $v_l$, $v_r$

**Key limitation**: Position $(x, y)$ is **not directly measurable**. The Kalman filter must integrate velocity information to estimate position, which will accumulate drift over time.

Measurement vector: $\mathbf{y} = [\theta, \dot{\theta}, v_l, v_r]^T$

$$C = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & -1/L & 1/L \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix}, \quad D = 0$$

In [ ]:
A = np.array([
    [0, 0,   0,      0.5,   0.5],
    [0, 0,   0.5*v0,  0.0,   0.0],
    [0, 0,   0,     -1/L,   1/L],
    [0, 0,   0,  -1.0/tau,  0.0],
    [0, 0,   0,      0.0, -1.0/tau],
])

B = np.array([
    [0.0,      0.0],
    [0.0,      0.0],
    [0.0,      0.0],
    [1.0/tau,  0.0],
    [0.0,  1.0/tau],
])

# Measurement model: [theta, theta_dot, v_l, v_r]
C = np.array([
    [0, 0, 1,     0,    0],      # magnetometer: theta
    [0, 0, 0, -1/L, 1/L],        # gyro: theta_dot
    [0, 0, 0,     1,    0],      # left encoder: v_l
    [0, 0, 0,     0,    1],      # right encoder: v_r
])
D = np.zeros((n_outputs, n_inputs))

sys_ct = ctl.ss(A, B, C, D)

### Discretization

Convert the continuous-time system to discrete-time using zero-order hold (ZOH) method:

$$\mathbf{x}_{k+1} = A_d \mathbf{x}_k + B_d \mathbf{u}_k$$
$$\mathbf{y}_k = C_d \mathbf{x}_k + D_d \mathbf{u}_k$$

This allows us to simulate the system at a fixed sample rate (50 Hz, or $\Delta t = 0.02$ s).

In [ ]:
Ad, Bd, Cd, Dd = discretize_system(A, B, C, D, dt)

### Kalman Filter with Realistic Sensors

The Kalman filter provides optimal state estimation in the presence of Gaussian noise. It operates in two steps:

#### Predict Step
Propagate the state and covariance forward in time:
$$\hat{\mathbf{x}}_{k|k-1} = A_d \hat{\mathbf{x}}_{k-1|k-1} + B_d \mathbf{u}_{k-1}$$
$$P_{k|k-1} = A_d P_{k-1|k-1} A_d^T + Q$$

#### Update Step
Incorporate the measurement to refine the estimate:
$$K_k = P_{k|k-1} C_d^T (C_d P_{k|k-1} C_d^T + R)^{-1}$$
$$\hat{\mathbf{x}}_{k|k} = \hat{\mathbf{x}}_{k|k-1} + K_k (\mathbf{y}_k - C_d \hat{\mathbf{x}}_{k|k-1})$$
$$P_{k|k} = (I - K_k C_d) P_{k|k-1}$$

The Kalman filter optimally balances predictions from the dynamics model with noisy measurements.

#### Position Estimation Challenge

Since position $(x, y)$ cannot be directly measured, the filter must **integrate** wheel velocities over time:
- Position estimates rely purely on the **predict step** (dynamics propagation)
- Measurements can only correct heading and wheel speeds
- **Position uncertainty grows unbounded** over time (dead reckoning drift)
- In practice, this requires periodic position updates from GPS, vision, or other external references

## LQR Controller Design

### Linear Quadratic Regulator (LQR) Theory

LQR is an optimal control technique that minimizes a quadratic cost function of the form:

$$J = \sum_{k=0}^{\infty} \left( \mathbf{x}_k^T Q_{lqr} \mathbf{x}_k + \mathbf{u}_k^T R_{lqr} \mathbf{u}_k \right)$$

where:
- $Q_{lqr}$: State cost matrix (penalizes deviation from desired state)
- $R_{lqr}$: Control cost matrix (penalizes control effort)

The optimal control law is:

$$\mathbf{u}_k = -K_{lqr} \mathbf{x}_k$$

where $K_{lqr}$ is the LQR gain matrix computed by solving the discrete-time algebraic Riccati equation (DARE).

### Design Trade-offs

- **High $Q_{lqr}$**: Prioritizes tracking accuracy, may result in aggressive control
- **High $R_{lqr}$**: Prioritizes smooth control inputs, may result in slower response
- The ratio between $Q_{lqr}$ and $R_{lqr}$ determines the performance/effort trade-off

### Reference Tracking

For tracking a reference state $\mathbf{x}_{ref}$, we use error-based control:

$$\mathbf{u}_k = -K_{lqr} (\mathbf{x}_k - \mathbf{x}_{ref})$$

This drives the robot toward the desired position and heading.

In [ ]:
# Initial conditions
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
P_init = np.eye(n_states) * P0

In [ ]:
# Initialize LQR controller with reference switching
lqr = SwitchingLQRController(
    Ad, Bd, Q_lqr, R_lqr,
    x_ref_pos=x_ref_pos,
    x_ref_stop=x_ref_stop,
    target_pos=[x_target, y_target],
    target_threshold=target_threshold,
    u_max=u_max
)

print("LQR Gain Matrix K:")
print(lqr.get_gain())
print("\nClosed-loop eigenvalues:")
print(lqr.get_closed_loop_eigenvalues())

## Simulation with LQR Control

### Scenario Setup

The simulation demonstrates trajectory tracking with LQR feedback control:
1. **Goal**: Navigate to target position $(x=5, y=3)$ with heading $\theta=0$
2. **Initial state**: Robot starts at origin with zero velocity
3. **Control law**: $\mathbf{u} = -K_{lqr}(\mathbf{x} - \mathbf{x}_{ref}) + \mathbf{u}_{ff}$
   - Feedback term drives the error to zero
   - Optional feedforward term $\mathbf{u}_{ff}$ can maintain desired wheel speeds

### Simulation Components

The simulation includes:
- **True system**: Evolves with process noise and LQR control input
- **Measurements**: Noisy observations of the true state
- **Kalman filter**: Estimates the state from noisy measurements
- **LQR controller**: Uses the estimated state to compute control inputs

This closed-loop simulation demonstrates how LQR + Kalman filter work together for optimal control and estimation.

In [ ]:
# Initialize Kalman Filter
kf = KalmanFilter(Ad, Bd, Cd, Q, R, x0=x0, P0=P_init)

# Initialize LQR controller with reference switching
lqr = SwitchingLQRController(
    Ad, Bd, Q_lqr, R_lqr,
    x_ref_pos=np.array([5.0, 3.0, 0.0, v0, v0]),
    x_ref_stop=np.array([5.0, 3.0, 0.0, 0.0, 0.0]),
    target_pos=[5.0, 3.0],
    target_threshold=0.5,
    u_max=2.0
)

# Initialize simulation
system_matrices = {'Ad': Ad, 'Bd': Bd, 'Cd': Cd}
sim = TankDriveSimulation(
    system_matrices=system_matrices,
    filter_obj=kf,
    controller_obj=lqr,
    Q=Q,
    R=R,
    dt=dt,
    N=N,
    random_seed=42
)

# Set initial conditions
sim.set_initial_conditions(x0_true=np.zeros(n_states), x0_hat=x0)

# Run simulation
results = sim.run()

print("Simulation complete!")

# Extract results
x_true = results['x_true']
x_hat = results['x_hat']
u_control = results['u_control']
P_hist = results['P_hist']
t = results['t']

## Results Visualization

### Plots Generated

1. **Trajectory plot**: Shows the robot's path in the x-y plane
   - True trajectory (with process noise)
   - Kalman filter estimate
   - Target position marked with red 'X'
   - Demonstrates convergence to the goal

2. **Position error time history**: Distance from target position
   - $e_{pos} = \sqrt{(x - x_{ref})^2 + (y - y_{ref})^2}$
   - Shows how quickly the LQR controller reduces position error

3. **State time histories**: Five subplots showing:
   - Position ($x$) convergence to target
   - Position ($y$) convergence to target
   - Heading angle ($\theta$) regulation
   - Left wheel speed ($v_l$) dynamics
   - Right wheel speed ($v_r$) dynamics
   - Dashed lines show reference values

4. **Control inputs**: Commanded wheel speeds over time
   - Shows the LQR controller's actuator commands
   - Demonstrates smooth control action with saturation if needed

The results should show the robot successfully navigating to the target position with the Kalman filter providing accurate state estimates despite measurement noise.

In [ ]:
# Plot 1: Trajectory in x-y plane
plt.figure(figsize=(10, 6))
plt.plot(x_true[0, :], x_true[1, :], 'b-', linewidth=2, label='True path')
plt.plot(x_hat[0, :], x_hat[1, :], 'g--', linewidth=2, label='KF estimate')
plt.plot(x_target, y_target, 'rx', markersize=15, markeredgewidth=3, label='Target')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.legend()
plt.title('LQR-Controlled Tank Drive Trajectory')

# Plot 2: Position error over time
plt.figure(figsize=(10, 4))
pos_error = np.sqrt((x_true[0, :] - x_target)**2 + (x_true[1, :] - y_target)**2)
plt.plot(t, pos_error, 'b-', linewidth=2)
plt.xlabel('time [s]')
plt.ylabel('Position error [m]')
plt.grid(True, alpha=0.3)
plt.title('Distance to Target')

# Plot 3: State time histories
fig, axes = plt.subplots(5, 1, figsize=(10, 12))

# Get current reference for plotting
x_ref = lqr.get_reference()

# X position
axes[0].plot(t, x_true[0, :], 'b-', label='True')
axes[0].plot(t, x_hat[0, :], 'g--', label='KF estimate')
axes[0].axhline(x_ref[0], color='r', linestyle=':', linewidth=2, label='Reference')
axes[0].set_ylabel('x [m]')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Y position
axes[1].plot(t, x_true[1, :], 'b-', label='True')
axes[1].plot(t, x_hat[1, :], 'g--', label='KF estimate')
axes[1].axhline(x_ref[1], color='r', linestyle=':', linewidth=2, label='Reference')
axes[1].set_ylabel('y [m]')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

# Heading
axes[2].plot(t, x_true[2, :], 'b-', label='True')
axes[2].plot(t, x_hat[2, :], 'g--', label='KF estimate')
axes[2].axhline(x_ref[2], color='r', linestyle=':', linewidth=2, label='Reference')
axes[2].set_ylabel('θ [rad]')
axes[2].grid(True, alpha=0.3)
axes[2].legend()

# Left wheel speed
axes[3].plot(t, x_true[3, :], 'b-', label='True')
axes[3].plot(t, x_hat[3, :], 'g--', label='KF estimate')
axes[3].axhline(x_ref[3], color='r', linestyle=':', linewidth=2, label='Reference')
axes[3].set_ylabel('v_l [m/s]')
axes[3].grid(True, alpha=0.3)
axes[3].legend()

# Right wheel speed
axes[4].plot(t, x_true[4, :], 'b-', label='True')
axes[4].plot(t, x_hat[4, :], 'g--', label='KF estimate')
axes[4].axhline(x_ref[4], color='r', linestyle=':', linewidth=2, label='Reference')
axes[4].set_ylabel('v_r [m/s]')
axes[4].set_xlabel('time [s]')
axes[4].grid(True, alpha=0.3)
axes[4].legend()

plt.tight_layout()

# Plot 4: Control inputs
plt.figure(figsize=(10, 5))
plt.subplot(2, 1, 1)
plt.plot(t[:-1], u_control[:-1, 0], 'b-', linewidth=2)
plt.ylabel('Left wheel cmd [m/s]')
plt.grid(True, alpha=0.3)
plt.title('LQR Control Inputs')

plt.subplot(2, 1, 2)
plt.plot(t[:-1], u_control[:-1, 1], 'r-', linewidth=2)
plt.ylabel('Right wheel cmd [m/s]')
plt.xlabel('time [s]')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Position Uncertainty Analysis

With realistic sensors (no direct position measurement), we can examine how position uncertainty grows over time. The Kalman filter's covariance matrix $P$ tracks this uncertainty.

In [ ]:
# Compute position error and statistics
pos_error = sim.compute_position_error()
stats = sim.compute_statistics()

# Position uncertainty growth
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(t, np.sqrt(P_hist[0, :]), 'b-', linewidth=2, label='σ_x')
plt.plot(t, np.sqrt(P_hist[1, :]), 'r-', linewidth=2, label='σ_y')
plt.ylabel('Position uncertainty [m]')
plt.grid(True, alpha=0.3)
plt.legend()
plt.title('Position Uncertainty Growth (Dead Reckoning)')

plt.subplot(2, 1, 2)
plt.plot(t, np.sqrt(P_hist[2, :]), 'g-', linewidth=2, label='σ_θ')
plt.plot(t, np.sqrt(P_hist[3, :]), 'm-', linewidth=2, label='σ_v_l')
plt.plot(t, np.sqrt(P_hist[4, :]), 'c-', linewidth=2, label='σ_v_r')
plt.ylabel('Uncertainty [rad or m/s]')
plt.xlabel('time [s]')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

# Print statistics
print(f"\nFinal position uncertainty (1σ): x={stats['final_position_uncertainty']['x']:.4f} m, y={stats['final_position_uncertainty']['y']:.4f} m")
print(f"Mean position error: {stats['position_error']['mean']:.4f} m")
print(f"Max position error: {stats['position_error']['max']:.4f} m")